In [1]:
# # install latest CyberGIS library
# !rm -rf /tmp/cybergis && \
#     rm -rf /opt/conda/envs/hydro-py3-new/lib/python3.7/site-packages/cybergis && \
#     git clone https://github.com/cybergis/Jupyter-xsede.git --branch general /tmp/cybergis && \
#     cp -r /tmp/cybergis/cybergis /opt/conda/envs/hydro-py3-new/lib/python3.7/site-packages && \
#     cd ~ && \
#     rm -rf /tmp/cybergis && \
#     rm -rf /home/jovyan/work/Downloads/1f3f310af8364d2aa3e6a9459152a21c && \
#     cd /home/jovyan/work/workspace/ && rm -rf * && \
#     echo CyberGIS installed. Please restart Kernel.

In [1]:
import os
import sys
import logging
os.environ["USER"]=""
from cybergis.general import *
logger.setLevel(logging.INFO)

/Users/zhiyul/miniconda3/envs/cbgis_py3/lib/python3.7/site-packages/xarray/core/merge.py:17: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  PANDAS_TYPES = (pd.Series, pd.DataFrame, pd.Panel)


### Unzip model package

In [2]:

#Unzip model file
model_folder_name = "SummaModel_ReynoldsAspenStand_StomatalResistance_sopron"
#instance = utils.download_model_instance(resource_id)
model_source_folder_path = os.path.abspath (model_folder_name)
file_manger_rel_path = "settings/summa_fileManager_riparianAspenSimpleResistance.txt"

### "Install" the model on Jupyter server (fix broken path in summa file_manager)

In [3]:

!cd {model_source_folder_path} && chmod +x ./installTestCases_local.sh
!cd {model_source_folder_path} && ./installTestCases_local.sh

TestCases installed


### Use PySumma to configure model parameters

In [4]:
from pysumma import Simulation

# path to the SUMMA filemanager file
file_manager = os.path.join(model_source_folder_path, file_manger_rel_path)

# Create pySUMMA Simulation Object
executable = ""
S = Simulation(executable, file_manager)
#print(S)
S.decisions['simulStart'].value = "2006-07-01 00:00"
S.decisions['simulFinsh'].value = "2007-08-20 00:00"
S.decisions['stomResist'].value = 'BallBerry'
S._write_configuration() # save configiuration to disk

### Submit model to HPC resources using CyberGIS-Jupyter tool

In [11]:
from cybergis.general import summaUI
local_workspace_path = os.path.abspath("./workspace")
! mkdir -p {local_workspace_path}
private_key=os.path.abspath("./keeling.key")
s = summaUI(model_source_folder_path, file_manger_rel_path, local_workspace_path, private_key_path=private_key)

In [12]:
s.runSumma()

2019-10-10 15:54:21,151 summaUI Obj: 13491587152
2019-10-10 15:54:21,152 btn obj: 13487938064
2019-10-10 15:54:21,203 SSH logged into keeling.earth.illinois.edu as user cigi-gisolve
2019-10-10 15:54:21,207 Uploading /Users/zhiyul/Documents/Projects/summa/workspace/Summa_1570740861_36e44f7c to /data/keeling/a/cigi-gisolve
2019-10-10 15:54:24,610 Submitting Job sbatch.sh to queue
2019-10-10 15:54:29,006 R
2019-10-10 15:54:32,693 R
2019-10-10 15:54:41,385 R
2019-10-10 15:54:45,022 Job completed: Summa_1570740861_36e44f7c; 3148563
2019-10-10 15:54:45,024 Downloading /data/keeling/a/cigi-gisolve/Summa_1570740861_36e44f7c/SummaModel_ReynoldsAspenStand_StomatalResistance_sopron/output to /Users/zhiyul/Documents/Projects/summa/workspace/Summa_1570740861_36e44f7c
2019-10-10 15:54:47,318 Downloading /data/keeling/a/cigi-gisolve/Summa_1570740861_36e44f7c/SummaModel_ReynoldsAspenStand_StomatalResistance_sopron/slurm-3148563.out to /Users/zhiyul/Documents/Projects/summa/workspace/Summa_1570740861_3

### Visualization

In [10]:
s= None

In [ ]:
summa_output = os.path.join(local_workspace_path, 
                            "{}/output".format(s.job_local_id), 
                            "vegImpactsTranspire_output_test_timestep.nc")



In [ ]:
import xarray as xr
output_info = xr.open_dataset(summa_output)
output_info

In [ ]:
import matplotlib.pyplot as plt
x = output_info['time']
y = (output_info['scalarCanopyTranspiration'] + \
     output_info['scalarCanopyEvaporation'] + \
     output_info['scalarGroundEvaporation'])*3600

plt.figure(figsize=(15, 5))
plt.plot(x, y, color='grey', linestyle='solid', markersize=0)

    # Get the current axis of the plot and
    # set the x and y-axis labels
ax = plt.gca()
ax.set_ylabel("Total ET (kg m-2 h-1)")
ax.set_xlabel('Date')
ax.grid(True)

ax.set_title('SUMMA Plot for Total ET')
output_info.close()